In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,recall_score,confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#dataSet = pd.read_csv('gender-classifier.csv',sep=',')
dataSet=pd.read_csv('gender-classifier.csv', sep=',', encoding='latin-1')
dataSet.head(2)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)


In [3]:
#dropping some of the reduntant column
dataSet.drop(['_golden','_unit_id','description','_last_judgment_at','created','retweet_count','text','profileimage','sidebar_color','tweet_coord','tweet_count','tweet_created','tweet_id','tweet_location','user_timezone'],axis=1,inplace=True)

In [4]:
dataSet.fillna(method='bfill',inplace=True)
dataSet.fillna(method='ffill',inplace=True)

In [5]:
from sklearn.preprocessing import LabelEncoder
target=dataSet['gender']
target=LabelEncoder().fit_transform(target)


In [6]:
#dropping target label and encoding catagorical values
dataSet.drop('gender',axis=1,inplace=True)

dataSet['_unit_state']=LabelEncoder().fit_transform(dataSet['_unit_state'])
dataSet['profile_yn']=LabelEncoder().fit_transform(dataSet['profile_yn'])
dataSet['gender_gold']=LabelEncoder().fit_transform(dataSet['gender_gold'])
dataSet['name']=LabelEncoder().fit_transform(dataSet['name'])
dataSet['profile_yn_gold']=LabelEncoder().fit_transform(dataSet['profile_yn_gold'])
dataSet['link_color']=LabelEncoder().fit_transform(dataSet['link_color'])
dataSet.head()

,_unit_state,_trusted_judgments,gender:confidence,profile_yn,profile_yn:confidence,fav_number,gender_gold,link_color,name,profile_yn_gold
0,0,3,1.0000,1,1.0,0,3,248,17159,0
1,0,3,1.0000,1,1.0,68,3,77,3229,0
2,0,3,0.6625,1,1.0,7696,3,2013,15258,0
3,0,3,1.0000,1,1.0,202,3,77,12732,0
4,0,3,1.0000,1,1.0,37318,3,1019,10222,0


In [7]:
    #train and test data set spilitting
    X = dataSet
    y=target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [8]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [10]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("accuracy{:.3f}".format(accuracies.mean()))
#accuracies.std()

accuracy0.520


In [11]:
# Making the Confusion Matrix
a=confusion_matrix(y_test, y_pred)
a

array([[1119,  262,  461,   30],
       [ 307, 1131,  513,   12],
       [ 307,  649,  894,   23],
       [ 118,   55,  106,   28]])